In [1]:
#Initialize
from factfinder.calculate import Calculate
from factfinder.special import pivot
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd

# pd.options.display.float_format = "{:,.18f}".format

try:
    env_path = "../.env"
    load_dotenv(dotenv_path=env_path)
except:
    print(".env file is missing ...")

calculate = Calculate(api_key=os.environ["API_KEY"], year=2019, source="acs", geography='2010_to_2020')

In [2]:
##INPUTS -- change here
pff_variable = 'mntrvtm'
geotype = 'CT20'
census_geoid_list = ['36047054300']

In [3]:
# See all digits of ratio
ratio = calculate.geo.ratio
ratio.loc[ratio.geoid_ct2020.isin(census_geoid_list), :]


,geoid_ct2010,geoid_ct2020,ratio
877,36047054300,36047054300,0.021407


In [4]:
# Get special base variables
special_base_vars = calculate.meta.get_special_base_variables(pff_variable)


In [5]:
# Perfom e and m calculation by calling calculate method
df = calculate.calculate_e_m_special(pff_variable, geotype)
df.head()


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.census.gov:443
DEBUG:urllib3.connectionpool:https://api.census.gov:443 "GET /data/2019/acs/acs5?get=NAME%2CB08013_001E%2CB08013_001M&for=tract%3A%2A&key=1676c02d269f644eea2083e6ed81666f7afb8f5d&in=state%3A36+county%3A005 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.census.gov:443 "GET /data/2019/acs/acs5/variables/NAME.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.census.gov:443 "GET /data/2019/acs/acs5/variables/B08013_001E.json HTTP/1.1" 200 290
DEBUG:urllib3.connectionpool:https://api.census.gov:443 "GET /data/2019/acs/acs5/variables/B08013_001M.json HTTP/1.1" 200 344
DEBUG:urllib3.connectionpool:https://api.census.gov:443 "GET /data/2019/acs/acs5/variables/state.json HTTP/1.1" 404 133
DEBUG:urllib3.connectionpool:https://api.census.gov:443 "GET /data/2019/acs/acs5/variables/county.json HTTP/1.1" 404 134
DEBUG:urllib3.connectionpool:https://api.census.gov:443 "GET /data/2019/acs/acs

,census_geoid,pff_variable,geotype,e,m
0,36005000100,mntrvtm,CT20,NaN,NaN
1,36005000200,mntrvtm,CT20,43.394616,12.976134
2,36005000400,mntrvtm,CT20,52.868440,9.347616
3,36005001600,mntrvtm,CT20,47.987897,10.395833
4,36005001901,mntrvtm,CT20,42.078775,19.969490


In [6]:
# Calculate inputs
inputs_df = calculate.calculate_e_m_multiprocessing(special_base_vars, geotype)
inputs_df.loc[inputs_df.census_geoid.isin(census_geoid_list),:]

,census_geoid,pff_variable,geotype,e,m
854,36047054300,agttm,CT20,150.489297,310.662308
854,36047054300,wrkr16pl,CT20,4.987768,10.560063
854,36047054300,cw_wrkdhm,CT20,0.642202,3.069786
854,36047054300,wrkrnothm,CT20,4.345566,10.997205


In [7]:
def mntrvtm(df: pd.DataFrame, base_variables: list) -> pd.DataFrame:
    df = pivot(df, base_variables)
    df["e"] = df["agttme"] / (df["wrkr16ple"] - df["cw_wrkdhme"])
    df["m"] = (
        1
        / df["wrkrnothme"]
        * np.sqrt(
            df["agttmm"] ** 2
            + (df["agttme"] * df["wrkrnothmm"] / df["wrkrnothme"]) ** 2
        )
    )
    return df

In [8]:
# Apply function to inputs
df = mntrvtm(inputs_df, special_base_vars)
df["pff_variable"] = pff_variable
df["geotype"] = geotype
df.loc[df.census_geoid.isin(census_geoid_list),:]

,census_geoid,agttme,agttmm,wrkr16ple,wrkr16plm,cw_wrkdhme,cw_wrkdhmm,wrkrnothme,wrkrnothmm,e,m,pff_variable,geotype
854,36047054300,150.489297,310.662308,4.987768,10.560063,0.642202,3.069786,4.345566,10.997205,34.630542,113.098481,mntrvtm,CT20


In [9]:
# Peform full calculation (including cleaning/rounding) to show display output
full_calc = calculate(pff_variable, geotype)
full_calc.loc[full_calc.census_geoid.isin(census_geoid_list),:]

,census_geoid,labs_geoid,geotype,labs_geotype,pff_variable,c,e,m,p,z
854,36047054300,3054300,CT20,CT2020,mntrvtm,198.5,34.6,113.1,100.0,NaN
